# **Pure $\lambda$-Calculus**

[The deatailed script](https://www.mathcha.io/editor/Pvvz5UZ1t7ktL6sZJYp19sZnX9vVserJMEKhJvvMx7)

## **Variables**

The code below models variables.

Using the `natgen()` generator in this code ensures that a fresh variable is returned in response to each constructor call.

In [1]:
def natgen():
    n = 0
    while True:
        yield n
        n += 1


class Var:
    __nats = natgen()

    def __init__(self):
        self._idx = next(Var.__nats)

    def __hash__(self):
        return self._idx.__hash__()

    def __str__(self):
        return "v[" + str(self._idx) + "]"

    def __eq__(self, other):
        return self._idx == other._idx

## **Terms**


In [2]:
class Term:

    @property
    def isAtom(self):
        """checks whether the term is an atom"""
        return isinstance(self, Atom)

    @property
    def isApplication(self):
        """checks whether the term is an application"""
        return isinstance(self, Application)

    @property
    def isAbstraction(self):
        """checks whether the term is an abstraction"""
        return isinstance(self, Abstraction)

    def __str__(self):
        if self.isAtom:
            return str(self._var)
        if self.isApplication:
            return "(" + str(self._sub) + " " + str(self._obj) + ")"
        # self is Abbstraction
        return "(fun " + str(self._head) + " => " + str(self._body) + ")"

    def __eq__(self, other):
        if self.isAtom and other.isAtom:
            return self._var == other._var
        if isinstance(self, Application) and isinstance(other, Application):
            return self._sub == other._sub and self._obj == other._obj
        if isinstance(self, Abstraction) and isinstance(other, Abstraction):
            return self._head == other._head and self._body == other._body

    @property
    def isBetaRedex(self):
        """checks whether the term is a beta-redex"""
        return self.isApplication and self._sub.isAbstraction

    @property
    def redexes(self):
        """determiness all beta-redexes in the term"""
        if self.isAtom:
            return []
        if self.isAbstraction:
            return self._body.redexes
        # self is Application
        temp = [self ]if self.isBetaRedex else []
        temp += (self._sub.redexes + self._obj.redexes)
        return temp


    @property
    def _vars(self):
        """
        returns
        -------
            the dictionary stuctured as follows
                dict[Var, dict[['free' | 'bound'], int]]
            Here, keys of the external dictionary are the variables that
            are occurred in 'self', and values of the internal dictionaries
            relate respectively to the numbers of free and bound occurrences
            of the variables.
        """
        if self.isAtom:
            return {self._var: {'free': 1, 'bound': 0}}
        if self.isApplication:
            vars, auxvars = dict(self._sub._vars), self._obj._vars
            for var in auxvars:
                try:
                    for key in {'free', 'bound'}:
                       vars[var][key] += self._obj._vars[var][key]
                except KeyError:
                    vars[var] = dict(self._obj._vars[var])
            return vars
        # self is Abstraction
        vars = dict(self._body._vars)
        try:
            vars[self._head]['bound'] += vars[self._head]['free']
            vars[self._head]['free'] = 0
        except KeyError:
            pass
        return vars

    @property
    def verticesNumber(self):
      """return the number of nodes in the tree representing the lambda term"""
      if self.isAtom:
        return 1
      elif self.isApplication:
        return 1 + self._sub.verticesNumber + self._obj.verticesNumber
      else: # self is Abstraction
        return 1 + self._body.verticesNumber 

    def hasNormalForm(self, strategy):
      """
      :param strategy: OneStepStrategy
      """
      term = self._updateBoundVariables()
      count = 0
      while term.redexes != []:
        term = term._betaConversion(strategy)
        count += 1
        if term.verticesNumber > 100000 or count > 2000:
          return False
      return True    

    def normalize(self, strategy):
      """
      :param strategy: OneStepStrategy
      :return tuple of the normal form of the term and number of steps of betta reduction
      """
      term = self._updateBoundVariables()
      count = 0
      while term.redexes != []:
        term = term._betaConversion(strategy)
        count += 1
      return (term, count)

    def _betaConversion(self, strategy):
      """
      :param strategy: OneStepStrategy
      :return term with redex eliminated using the given strategy
      """
      index = strategy.redexIndex(self)
      subterm = self.subterm(index)
      reducedTerm = subterm._removeOuterRedex()
      return self.setSubterm(index, reducedTerm)

    def subterm(self, index: int):
      """
      By representing the term as a tree, a subtree is returned, which is also a lambda term.
      The vertex of this subtree has a given index in the topological sorting of the vertices of the original term.
      :param index - subterm index
      :return: subterm: Term
      """
      if index == 1:
        return self

      if self.isAtom:
        ValueError('index value is incorrect')
      elif self.isApplication:
        if self._sub.verticesNumber + 1 >= index:
          return self._sub.subterm(index - 1)
        else:
          return self._obj.subterm(index - self._sub.verticesNumber - 1)
      else: # self is Abstraction
        return self._body.subterm(index - 1)

    def setSubterm(self, index: int, term):
      """
      By representing the term as a tree, a subtree is set, which is also a lambda term.
      The vertex of this subtree has a given index in the topological sorting of the vertices of the original term.
      :param index - subterm index
      :param term - λ-term to which the subterm will be replaced
      :return: updated λ-term
      """
      if index == 1:
        return term

      if self.isAtom:
        ValueError('index value is incorrect')
      elif self.isApplication:
        if self._sub.verticesNumber + 1 >= index:
          return Application(self._sub.setSubterm(index - 1, term), self._obj)
        else:
          return Application(self._sub, self._obj.setSubterm(index - self._sub.verticesNumber - 1, term))
      else: # self is Abstraction
        return Abstraction(self._head, self._body.setSubterm(index - 1, term))

    def _updateBoundVariables(self):
      """return λ-term with updated bound variables"""
      if self.isAtom:
        return self
      elif self.isApplication:
        return Application(self._sub._updateBoundVariables(), self._obj._updateBoundVariables())
      else: # self is Abstraction
        newVar = Var()
        return Abstraction(newVar, self._body._replaceVariable(self._head, Atom(newVar))._updateBoundVariables())

    def _removeOuterRedex(self):
      """apply the betta conversion to the lambda term, removing the outer betta redex"""
      if self.isBetaRedex:
        head = self._sub._head
        body = self._sub._body
        return body._replaceVariable(head, self._obj)
      else:
        return self

    def _replaceVariable(self, var: Var, term):
      """return λ-term with replaced variable"""
      if self.isAtom:
        return term if self._var == var else self
      elif self.isApplication:
        return Application(self._sub._replaceVariable(var, term), self._obj._replaceVariable(var, term))
      else: # self is Abstraction
        return Abstraction(self._head, self._body._replaceVariable(var, term))


class Atom(Term):
    def __init__(self, x: Var):
        if isinstance(x, Var):
            self._var = x
        else:
            raise TypeError("a variable is waiting")


class Application(Term):
    def __init__(self, X : Term, Y : Term):
        if isinstance(X, Term) and isinstance(Y, Term):
            self._sub = X
            self._obj = Y
        else:
            raise TypeError("a term is waiting")


class Abstraction(Term):
    def __init__(self, x: Var, X: Term):
        if isinstance(x, Var):
            if isinstance(X, Term):
                self._head = x
                self._body = X
            else:
                raise TypeError("a term is waiting")
        else:
            raise TypeError("a variable is waiting")


## Strategy


In [3]:
from abc import ABC, abstractmethod

class OneStepStrategy(ABC):
    
  @abstractmethod
  def redexIndex(self, term: Term, initIndex = 0) -> int:
      """
      :return: index of the vertex of a subterm that has an outer redex. 
              The index of a vertex is the index of this vertex in the topological sort of the tree vertices.
              Indexing starts at 1.
      """

class LeftmostOutermostStrategy(OneStepStrategy):

  def redexIndex(self, term: Term, initIndex = 0) -> int:
    if term.isAtom or len(term.redexes) == 0:
      ValueError('the term does not contain a redex')
    elif term.isApplication:
      if term.isBetaRedex:
        return initIndex + 1
      elif len(term._sub.redexes) != 0:
        return self.redexIndex(term._sub, initIndex + 1)
      else:
        return self.redexIndex(term._obj, initIndex + term._sub.verticesNumber + 1)
    else: # self is Abstraction
      return self.redexIndex(term._body, initIndex + 1)

## Generating lambda terms

In [9]:
import random
from typing import List
import sys

sys.setrecursionlimit(20000)

def genTerm(p: float, vars: List[Var] = []) -> Term:

  pVar = (1 - p * p) / 2
  pAbs = pVar + p * p

  rand = random.random()

  if rand < pVar and len(vars) > 0:
    index = random.randint(0, len(vars) - 1)
    return Atom(vars[index]) 
  elif rand < pAbs:
    head = Var()
    return Abstraction(head, genTerm(p, vars + [head]))
  else:
    return Application(genTerm(p, vars), genTerm(p, vars))

def filterTerms(term):
  return 500 < term.verticesNumber < 8000 and term.hasNormalForm(LeftmostOutermostStrategy())

terms = list(filter(filterTerms, [genTerm(0.5093) for i in range(40)]))

if len(terms) != 0:
  print(list(map(lambda term: term.verticesNumber, terms)))
  print("mean number of vertices = {}".format(sum(map(lambda term: term.verticesNumber, terms)) / len(terms)))
  print("max number of vertices = {}".format(max(map(lambda term: term.verticesNumber, terms))))

  countRedexes = list(map(lambda term: len(term.redexes), terms))

  print(countRedexes)
  print("mean number of redexes = {}".format(sum(countRedexes) / len(countRedexes)))
  print("max number of redexes = {}".format(max(countRedexes)))

for term in terms:
  t, count = term.normalize(LeftmostOutermostStrategy())
  print(count)

[2448, 1218]
mean number of vertices = 1833.0
max number of vertices = 2448
[257, 112]
mean number of redexes = 184.5
max number of redexes = 257
19
1


## Tests

In [10]:
x, y, z = Var(), Var(), Var()
X, Z = Atom(x), Atom(z)
XXX = Application(Application(X, X), X)
XZ = Application(X, Z)
T = Application(Abstraction(x, XXX),
                Abstraction(x, Application(Abstraction(y, Z),
                                           XZ
                                          ))
               )

print(T)
for var, item in T._vars.items():
    print("\t{}".format(var), end=": ")
    print(item)

((fun v[230205] => ((v[230205] v[230205]) v[230205])) (fun v[230205] => ((fun v[230206] => v[230207]) (v[230205] v[230207]))))
	v[230205]: {'free': 0, 'bound': 4}
	v[230207]: {'free': 2, 'bound': 0}


In [11]:
x, y, z, w, v = Var(), Var(), Var(), Var(), Var()
# (λx.(λy.( ((λz.(y z)) ((λw.w) x)) v )))
lambdaTerm = Abstraction(x,
                        Abstraction(y,
                                    Application(
                                        Application(
                                            Abstraction(z, Application(Atom(y), Atom(z))),
                                            Application(Abstraction(w, Atom(w)), Atom(w))),
                                        Atom(v))))

def testTerm():
  assert(len(lambdaTerm.redexes) == 2)
  assert(lambdaTerm.verticesNumber == 13)

  subterm = Application(Atom(y), Atom(z))
  assert(lambdaTerm.subterm(1) == lambdaTerm)
  assert(lambdaTerm.subterm(6) == subterm)
  assert(lambdaTerm.setSubterm(1, subterm) == subterm)

  assert(lambdaTerm._updateBoundVariables().verticesNumber == lambdaTerm.verticesNumber)
  assert(len(lambdaTerm._updateBoundVariables().redexes) == len(lambdaTerm.redexes))

  strategy = LeftmostOutermostStrategy()
  assert(len(lambdaTerm._betaConversion(strategy).redexes) == 1)
  assert(lambdaTerm._betaConversion(strategy).verticesNumber == 10)

  assert(len(lambdaTerm.normalize(strategy)[0].redexes) == 0)
  assert(lambdaTerm.normalize(strategy)[1] == 2)


def testStrategy():
  strategy = LeftmostOutermostStrategy()
  assert(strategy.redexIndex(lambdaTerm) == 4)

testTerm()
testStrategy()